1. Min-cost max-flow

In [ ]:
import heapq
INF = 10 ** 9

'''struct Edge {
    int to;     // конечная вершина ребра
    int rev;    // индекс обратного ребра в списке смежности конечной вершины
    int capacity;    // остаточная пропускная способность
    int cost;   // стоимость единицы потока
    int flow;   // текущий поток
};'''

class MinCostMaxFlow:
    def __init__(self):
       self.n = 0
       self.graph = {}
       self.negative_cost = False

    def add_edge(self, u, v, capacity, cost):
        if cost < 0:
            self.negative_cost = True
        edge_v = {"to": v, "rev": len(self.graph.get(v, [])), "capacity": capacity, "cost": cost, "flow": 0}
        if u in self.graph:
            self.graph[u].append(edge_v)
        else:
            self.graph[u] = [edge_v]
        edge_u = {"to": u, "rev": len(self.graph.get(u, [])) - 1, "capacity": 0, "cost": -cost, "flow": 0}
        if v in self.graph:
            self.graph[v].append(edge_u)
        else:
            self.graph[v] = [edge_u]

        self.n = max(list(self.graph.keys())) + 1


    def bellman_ford(self, start):
        pot = [INF] * self.n;
        pot[start] = 0;
        for i in range(1, self.n):
            for u in range(self.n):
                if (pot[u] == INF):
                    continue
            for edge in self.graph[u]:
                if edge["capacity"] > 0 and pot[edge["to"]] > pot[u] + edge["cost"]:
                    pot[edge["to"]] = pot[u] + edge["cost"]
        return pot

    def dijkstra(self, start, pot, prevv, preve):
        dist = [INF] * self.n
        dist[start] = 0

        pq = []
        heapq.heappush(pq, (0, start))

        while len(pq) > 0:
            d, u = heapq.heappop(pq)
            if d > dist[u]:
                continue

            for i, edge in enumerate(self.graph[u]):
                if (edge["capacity"] <= 0):
                    continue
                new_dist = dist[u] + edge["cost"] + pot[u] - pot[edge["to"]]
                if (dist[edge["to"]] > new_dist):
                    dist[edge["to"]] = new_dist
                    prevv[edge["to"]] = u
                    preve[edge["to"]] = i;
                    heapq.heappush(pq, (new_dist, edge["to"]))
        return dist, prevv, preve


    def solve(self, start, finish, max_flow=INF):
        total_flow, total_cost = 0, 0
        # Инициализация потенциалов при наличии отрицательных стоимостей
        # Если в графе есть ребра с отрицательной стоимостью,
        # используем алгоритм Беллмана-Форда
        # В противном случае можно инициализировать нулями

        if self.negative_cost:
            pot = self.bellman_ford(start)
        else:
            pot = [0] * self.n

        prevv = [0] * self.n
        preve = [0] * self.n

        while total_flow < max_flow:
            # Поиск кратчайшего пути с помощью Дейкстры
            dist, prevv, preve = self.dijkstra(start, pot, prevv, preve)

            # сток недостижим
            if dist[finish] == INF:
                break
            # Обновление потенциалов
            for i in range(self.n):
                if dist[i] < INF:
                    pot[i] += dist[i]
            # Нахождение минимального потока на пути
            flow = max_flow - total_flow
            v = finish
            while v != start:
                edge = self.graph[prevv[v]][preve[v]]
                flow = min(flow, edge["capacity"])
                v = prevv[v]
            # Обновление потока и стоимости
            v = finish
            while v != start:
                edge = self.graph[prevv[v]][preve[v]]
                edge["capacity"] -= flow
                self.graph[edge["to"]][edge["rev"]]["capacity"]+=flow
                total_cost += flow * edge["cost"]
                v = prevv[v]

            total_flow += flow
        return total_flow, total_cost

In [ ]:
# Создаем граф с 8 вершинами (0 - исток, 7 - сток)

graph = MinCostMaxFlow();

# Добавляем рёбра: from, to, capacity, cost
graph.add_edge(0, 1, 15, 2);
graph.add_edge(0, 2, 10, 3);
graph.add_edge(1, 3, 10, 2);
graph.add_edge(1, 4, 5, 4);
graph.add_edge(2, 3, 5, 1);
graph.add_edge(2, 5, 10, 2);
graph.add_edge(3, 6, 10, 3);
graph.add_edge(4, 6, 5, 2);
graph.add_edge(4, 7, 10, 1);
graph.add_edge(5, 7, 5, 3);
graph.add_edge(6, 7, 15, 2);

max_flow, min_cost = graph.solve(start=0, finish=7)

print(f"Максимальный поток: {max_flow}, Минимальная стоимость: {min_cost}")


Максимальный поток: 20, Минимальная стоимость: 165


2. Задача поиска компонент связности в динамически меняемом графе


In [ ]:
class Component:
    def __init__(self, comp_id):
        self.id = comp_id
        self.nodes = set()
        self.out_edges = set()  # компоненты, от которых есть ребро
        self.in_edges = set()   # компоненты, к которым есть ребро

    def add_node(self, node):
        self.nodes.add(node)

    def contains(self, node):
        return node in self.nodes

In [ ]:
class IncrementalSCC:
    def __init__(self, num_nodes):
        self.n = num_nodes
        self.next_component_id = 0
        self.node_to_component = [None] * num_nodes
        self.graph = [[] for _ in range(num_nodes)]

        # Изначально каждая вершина - отдельная компонента
        for i in range(num_nodes):
            comp = Component(self.next_component_id)
            self.next_component_id += 1
            comp.add_node(i)
            self.node_to_component[i] = comp

    # Поиск пути между компонентами в графе конденсации
    def has_path(self, from_comp, to_comp, visited):
        if from_comp == to_comp:
            return True
        if from_comp.id in visited:
            return False
        visited.add(from_comp.id)

        for neighbor in from_comp.out_edges:
            if self.has_path(neighbor, to_comp, visited):
                return True
        return False

    # Объединение компонент
    def merge_components(self, components_to_merge):
        new_component = Component(self.next_component_id)
        self.next_component_id += 1

        # Объединяем все вершины
        for comp in components_to_merge:
            for node in comp.nodes:
                new_component.add_node(node)
                self.node_to_component[node] = new_component

        # Объединяем входящие и исходящие рёбра
        all_in_edges = set()
        all_out_edges = set()

        for comp in components_to_merge:
            all_in_edges.update(comp.in_edges)
            all_out_edges.update(comp.out_edges)

        # Удаляем самоссылки
        all_in_edges.discard(new_component)
        all_out_edges.discard(new_component)

        # Обновляем рёбра
        new_component.in_edges = all_in_edges
        new_component.out_edges = all_out_edges

        # Обновляем ссылки у соседей
        for in_comp in new_component.in_edges:
            in_comp.out_edges.discard(new_component)
            in_comp.out_edges.add(new_component)

        for out_comp in new_component.out_edges:
            out_comp.in_edges.discard(new_component)
            out_comp.in_edges.add(new_component)

        return new_component

    # Вспомогательная функция для поиска компонент на пути
    def find_components_on_path(self, current, target, result, visited):
        if current.id in visited:
            return
        visited.add(current.id)

        result.append(current)

        if current == target:
            return

        for neighbor in current.out_edges:
            temp_visited = visited.copy()
            if self.has_path(neighbor, target, temp_visited):
                self.find_components_on_path(neighbor, target, result, visited)
                return

    # Добавление ребра
    def addEdge(self, u, v):
        self.graph[u].append(v)

        comp_u = self.node_to_component[u]
        comp_v = self.node_to_component[v]

        # Если вершины в одной компоненте - просто добавляем ребро
        if comp_u == comp_v:
            return

        # Добавляем ребро в граф конденсации
        comp_u.out_edges.add(comp_v)
        comp_v.in_edges.add(comp_u)

        # Проверяем, создаёт ли это ребро цикл
        visited = set()
        if self.has_path(comp_v, comp_u, visited):
            # Найден цикл - нужно объединить компоненты на пути
            components_to_merge = []
            visited2 = set()

            # Находим все компоненты на пути от comp_v к comp_u
            self.find_components_on_path(comp_v, comp_u, components_to_merge, visited2)

            # Объединяем компоненты
            new_component = self.merge_components(components_to_merge)

    # Получение компоненты для вершины
    def getComponent(self, node):
        return self.node_to_component[node].id

    # Вывод всех компонент
    def printComponents(self):
        printed = set()
        print("Компоненты сильной связности:")

        for i in range(self.n):
            comp = self.node_to_component[i]
            if comp not in printed:
                printed.add(comp)
                print(f"Компонента {comp.id}:", end=" ")
                for node in comp.nodes:
                    print(node, end=" ")
                print()

    # Вывод графа конденсации
    def printCondensationGraph(self):
        printed = set()
        print("Граф конденсации:")

        for i in range(self.n):
            comp = self.node_to_component[i]
            if comp not in printed:
                printed.add(comp)
                print(f"Компонента {comp.id} ->", end=" ")
                for out_comp in comp.out_edges:
                    if out_comp != comp:
                        print(out_comp.id, end=" ")
                print()

Исходное состояние (все вершины не связаны)

In [ ]:
scc = IncrementalSCC(num_nodes=4)
scc.printComponents()
scc.printCondensationGraph()

Компоненты сильной связности:
Компонента 0: 0 
Компонента 1: 1 
Компонента 2: 2 
Компонента 3: 3 
Граф конденсации:
Компонента 0 -> 
Компонента 1 -> 
Компонента 2 -> 
Компонента 3 -> 


Добавляем ребро 0 -> 1

In [ ]:
scc.addEdge(0, 1)
scc.printComponents()
scc.printCondensationGraph()

Компоненты сильной связности:
Компонента 0: 0 
Компонента 1: 1 
Компонента 2: 2 
Компонента 3: 3 
Граф конденсации:
Компонента 0 -> 1 
Компонента 1 -> 
Компонента 2 -> 
Компонента 3 -> 


Добавляем ребро 1 -> 2

In [ ]:
scc.addEdge(1, 2)
scc.printComponents()
scc.printCondensationGraph()

Компоненты сильной связности:
Компонента 0: 0 
Компонента 1: 1 
Компонента 2: 2 
Компонента 3: 3 
Граф конденсации:
Компонента 0 -> 1 
Компонента 1 -> 2 
Компонента 2 -> 
Компонента 3 -> 


In [ ]:
scc.addEdge(2, 0)
scc.printComponents()
scc.printCondensationGraph()

Компоненты сильной связности:
Компонента 4: 0 1 2 
Компонента 3: 3 
Граф конденсации:
Компонента 4 -> 1 2 0 
Компонента 3 -> 


In [ ]:
scc.addEdge(1, 3)
scc.printComponents()
scc.printCondensationGraph()

Компоненты сильной связности:
Компонента 4: 0 1 2 
Компонента 3: 3 
Граф конденсации:
Компонента 4 -> 3 1 2 0 
Компонента 3 -> 


In [ ]:
scc.addEdge(3, 2)
scc.printComponents()
scc.printCondensationGraph()

Компоненты сильной связности:
Компонента 5: 0 1 2 3 
Граф конденсации:
Компонента 5 -> 4 3 1 2 0 
